In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import time
import random
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from torch.multiprocessing.spawn import spawn
import torchvision.models as models
from torchvision.datasets import VOCSegmentation, CIFAR10
from torchvision.io import read_image
from torchvision.transforms import v2

In [7]:
def transform_image(img):
    transforms = v2.Compose([
        v2.ToTensor(), 
        v2.ToDtype(torch.float32, scale=True),
    ])
    return transforms(img)


class CustomDataset(Dataset):
    def __init__(self, is_train=True):
        self.dataset = CIFAR10(
            root="../data",
            train=is_train,
            download=False,
            transform=transform_image
        )

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        img = self.dataset[idx][0]
        img = transform_image(img)
        label = self.dataset[idx][1]
        return img, label

In [ ]:
# settings
epochs = 50
batch_size = 512

# seed
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# data
train_dataset = CustomDataset(is_train=True)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
val_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# model
model = models.vit_b_16(
    weights=None,
    image_size=32,
    num_classes=10)
# in_features = model.heads.head.in_features
# model.heads.head = nn.Linear(in_features, 10)
model = nn.DataParallel(model.to(device))

# others
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

# learning
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for imgs, labels in train_dataloader:
        optimizer.zero_grad()
        imgs, labels = imgs.to(device), labels.to(device) 
        preds = model(imgs)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, labels in val_dataloader:
            imgs, labels = imgs.to(device), labels.to(device) 
            preds = model(imgs)
            loss = criterion(preds, labels)
            val_loss += loss

    print(f"epoch: {epoch+1}, train_loss: {train_loss:.3f}, val_loss: {val_loss:.3f}")

ValueError: The parameter 'num_classes' expected value 1000 but got 10 instead.

In [ ]:
# 事前学習なし
# epoch: 50, train_loss: 134.397, val_loss: 132.811

